#  Multiple linear regression with statsmodels

# Note:
--> "Pandas" is mainly a package to handle and operate directly on data.

--> "Scikit-learn" is doing machine learning with emphasis on predictive modeling with often large and sparse data

--> "Statsmodels" is doing "traditional" statistics and econometrics, with much stronger emphasis on parameter estimation and (statistical) testing.

In [3]:
import statsmodels.api as sm

In [4]:
import pandas as pd

In [5]:
#Build a linear regression model from the provided data , loading the data.
my_dt=pd.read_csv("boston.csv")

In [6]:
#Check the top 5 rows of the imported data of all columns.
my_dt.head()

,CRIM,INDUS,NOX,RM,AGE,DIS,TAX,PT,B,MV
0,0.00632,2.31,0.538,6.575,65.199997,4.0900,296,15.300000,396.899994,24.000000
1,0.02731,7.07,0.469,6.421,78.900002,4.9671,242,17.799999,396.899994,21.600000
2,0.02729,7.07,0.469,7.185,61.099998,4.9671,242,17.799999,392.829987,34.700001
3,0.03237,2.18,0.458,6.998,45.799999,6.0622,222,18.700001,394.630005,33.400002
4,0.06905,2.18,0.458,7.147,54.200001,6.0622,222,18.700001,396.899994,36.200001


# Model_1:(my_model)

#  Split arrays or matrices into random train and test subsets

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#Splitting the data into train (75%) and test(25%) of dependent and independent variables with random_state(seed) as 999
#Provided:
        #y: Single column dataframe holding our solution vector for linear regression
        #x: A dataframe that runs parallel to y, with all the features for our linear regression
X_train,X_test,Y_train,Y_test=train_test_split(my_dt.drop("MV",axis=1),my_dt["MV"],test_size=0.25,random_state=999)

In [9]:
#add_constant: a boolean, if true it will add a constant row to our provided x data. Otherwise this method assumes you've done-so already, or do not want one for some good reason
#If add_constant was true this will be a new dataframe, otherwise it will be X_train
X_train = sm.add_constant(X_train)

In [10]:
#Train the Linear model using 'statsmodels.OLS' as 'sm.OLS'.
my_model = sm.OLS(Y_train, X_train)

In [11]:
#Model fit object obtained from a linear model trained using 'sm.OLS'.
result = my_model.fit()

In [12]:
#Entire summary of the model cosists of Constants,coef,std err, t , p value, R-squared,Adj R-squared ,AIC,BIC,etc
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                     MV   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     78.94
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           1.74e-80
Time:                        22:00:11   Log-Likelihood:                -1186.3
No. Observations:                 379   AIC:                             2393.
Df Residuals:                     369   BIC:                             2432.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.5247      6.386      3.214      0.0

In [13]:
# we need to import VIF to check or verify the multi collinearity between the variables 
from statsmodels.stats.outliers_influence import variance_inflation_factor
multicollinearity_model1=pd.Series([variance_inflation_factor(X_train.values, i) 
               for i in range(X_train.shape[1])], 
              index=X_train.columns)
print(multicollinearity_model1)

const    491.267613
CRIM       1.565644
INDUS      4.001190
NOX        4.048097
RM         1.347950
AGE        2.802511
DIS        3.496721
TAX        3.429197
PT         1.534356
B          1.273864
dtype: float64


# NOTE
The most apparent difference is that the VIFs are all down to satisfactory values, they are all less than 5. we can see that there is some multicollinearity in our data, but it is not severe enough to warrant further corrective measures.


#  Accuracy score test for model_1

In [14]:
# To calculate the accuracy score of the model we need to import sklearn.metrics.
from sklearn.metrics import r2_score

In [15]:
# Predicting the trained model with the test data 
predictions_model1 = result.predict(sm.add_constant(X_test))

In [16]:
# R-square value score of the prediction_model1 
r2_score(Y_test, predictions_model1)

0.6727577927397874

#   Significance of the predictors from the model_1 summary.

The p-value for each term tests the null hypothesis that the coefficient is equal to zero (no effect). A low p-value (< 0.05) indicates that you can reject the null hypothesis. In other words, a predictor that has a low p-value is likely to be a meaningful addition to your model because changes in the predictor's value are related to changes in the response variable.

In the output Above, we can see that the predictor variables of INDUS and TAX are insignificant because both of their p-values are above 0.05 which indicates that it is not statistically significant.

R-squared value     : 0.658 or 65.8%
Adj R-squared value : 0.650 or 65.0%
AIC                 : 2393.
BIC                 : 2432.

Accuracy of test score (R-score) : 67.2%

# Note: 
From the accuracy values of train and test models, test score accuracy is 1.5% more than train accuracy.so, we can say that our model is good trained with the data


# Model_2 :(Retrained_model)

In [17]:
# The goal of any variable selection is to removed unnecessary covariates with negligible contribution and 
# to removed correlated covariates so that the remaining covariates provide as much independent information 
# about the response as possible.
# Dropping the variables which have p-value of the exponential tail fit is insignificant which have value Greater than 0.05 

X_train=X_train.drop(["INDUS","TAX"],axis=1)


In [18]:
#Retrain the Linear model using 'sm.OLS' with only significant variables.
retrained_model = sm.OLS(Y_train, X_train)

In [19]:
#Model fit object obtained from a linear model retrained_model using 'sm.OLS'.
retrained_result=retrained_model.fit()

In [20]:
#Entire summary of the model cosists of Constants,coef,std err, t , p value, R-squared,Adj R-squared ,AIC,BIC,etc
print(retrained_result.summary())

                            OLS Regression Results                            
Dep. Variable:                     MV   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     101.5
Date:                Sun, 21 Apr 2019   Prob (F-statistic):           3.41e-82
Time:                        22:00:20   Log-Likelihood:                -1187.0
No. Observations:                 379   AIC:                             2390.
Df Residuals:                     371   BIC:                             2421.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.1223      6.338      3.175      0.0

In [21]:
# we need to import VIF to check or verify the multi collinearity between the variables 
from statsmodels.stats.outliers_influence import variance_inflation_factor
multicollinearity_model2=pd.Series([variance_inflation_factor(X_train.values, i) 
               for i in range(X_train.shape[1])], 
              index=X_train.columns)
print(multicollinearity_model2)

const    484.597701
CRIM       1.360539
NOX        3.176053
RM         1.278335
AGE        2.782281
DIS        3.230089
PT         1.276660
B          1.225698
dtype: float64


# NOTE
The most apparent difference is that the VIFs are all down to satisfactory values, they are all less than 5. we can see that there is some multicollinearity in our data, but it is not severe enough to warrant further corrective measures.

The VIF of the variables decreased compare to the model_1
 

# NOTE:Multi-Collinearity
Now, take a look at the Summary of Model tables for both models. You’ll notice that the standard error of the regression (S), R-squared, adjusted R-squared, and predicted R-squared are all identical. As I mentioned earlier, multicollinearity doesn’t affect the predictions or goodness-of-fit. If you just want to make predictions, the model with severe multicollinearity is just as good.

# Accuracy score test for model_2

In [22]:
# To calculate the accuracy score of the model we need to import sklearn.metrics.
from sklearn.metrics import r2_score

In [23]:
# We need to drop the variables which we actually dropped from the training data set which are insignificant
X_test = X_test.drop(["INDUS","TAX"], axis=1)

In [24]:
# Predicting the trained model with the test data 
prediction_model2 = retrained_result.predict(sm.add_constant(X_test))

In [25]:
# R-square value score of the prediction_model1 
r2_score(Y_test, prediction_model2)

0.6709891731985471

#   Significance of the predictors from the model_2 summary.

From the above model summary all the variables are significant where p value < 0.05.

R-squared score     = 0.657 or 65.7%

Adj R-squared score = 0.650 or 65.0%

AIC                 = 2390.

BIC                 = 2421.

**Adj R-squared remained constant in both the models as 65%

**Accuracy of test score (R-score) : 67.0%

# Note: 
From the accuracy values of train and test models, test score accuracy is 1.3% more than train accuracy.so, we can say that our model is good trained with the data



# Justified reason for dropping or not dropping a predictor?

From the above models with and without significants we observed :

**Dropping the insignificants variables every time is not a good idea and depends on the business requirement because they may be not totally independent with other variables.

**R-squared values of model_1=0.658(65.8%)and R-squared value of model_2=0.657(65.7%) . here, after dropping of 2 columns (INDUS,TAX) as they are not significant with respect to p-value .we can see the decrease in R-squared value of 0.1%.so, there are few factors influence on those two columns also. 

**Drop of columns should result in increasse in R-squared value but here there is a decrease in R-squared value of(0.1%)

**Accuracy R-square score on test data of both models (model-1=67.2%, model-2=67%) ,from this scores model_1 have 0.2%  higher Accuracy Score with test result than  model_2(after removing insignificant variables) test results.

**Adj R-squared remained constant in both the models as 65% 

**As there is decrease in AIC,BIC,Multi-collinearity also but they don't effect in prediction outputs.

**Finally, Model without not dropping the insignificant variable give more accurate predictor for new inputs.

 


